# Making Summary file and Rating files (Free-route)
1. rename cyoa spr .prolific to .txt files
2. grab from the spr file, the story map, and the prolific survey to make the summary file
3. compute additional information and output to another summary file; mannually paste to summary file.
4. create recall text files for processing and causality rating files (with story files).

In [1]:
#1. this is to change the .prolific files to .txt files
import xianfunc as xf

fpath = '../rate_importance/spr_data/'
filelist = xf.grab_all(fpath,'sona')#
print(len(filelist))
targlist = xf.replace_str(filelist,'.sona','.txt')
xf.rename_files(fpath,filelist,targlist)

filelist = xf.grab_all(fpath,'txt')
n_size = str(len(filelist))
print(n_size)

0
1


In [2]:
#2. make the summary file via grabing info from story map, cyoa spr file and the survey file

v = 4500 #uniq version number (alice-free=1000,alice-yoke=2000,alice-pasv=3000,monthy-free=4000)
sum_data = xf.init_dict(keylist=['sub','ID','pid','end_version','storyline','choice_vec','story_route','time_step','story','recall','story_word_count','recall_word_count'])

count=0
for file in filelist:
    print(file)
    count+=1
    completed = xf.grab_all('../rate_importance/spr_data/completed/','.txt')
    sub_num = count+len(completed)
    uniq_id = sub_num+v+len(completed)
    version = file.split('.')[0].split('_')[1]+ '_'+ file.split('.')[0].split('_')[2]
    subj_id = file.split('.')[0].split('_')[-1]

    data = xf.read_text(fpath+file,sep='\n',header='col1')
    spr_end = data['ending_version'][0]
    print(data['story_path'])
    if '13_2' in data['story_path']:
        spr_sl = 3
    elif '14_2' in data['story_path']:
        spr_sl = 2
    elif '14_1' in data['story_path']:
        spr_sl=1
    else:
        print('check story_path for storyline')
    spr_time_step = data['time_step'][1:]
    tstep = []
    for i in spr_time_step:
        if 'enter' in i or 'click' in i:
            tstep.append(float(i.split(':')[-1]))
    spr_time_step = ','.join([str(i) for i in tstep])
    spr_choice_vec = ','.join(data['choice_vec'])
    spr_story_path = ','.join(data['story_path'])
    spr_story_text = ' '.join(data['story_text'])
    spr_recall = ' '
    
    sum_data['sub'].append(str(sub_num))
    sum_data['ID'].append(str(uniq_id))
    sum_data['pid'].append(subj_id)
    sum_data['end_version'].append(str(spr_end))
    sum_data['storyline'].append(str(spr_sl))
    sum_data['choice_vec'].append(spr_choice_vec)
    sum_data['story_route'].append(spr_story_path)
    sum_data['time_step'].append(spr_time_step)
    sum_data['story'].append(spr_story_text)
    sum_data['recall'].append(spr_recall)
    sum_data['story_word_count'].append(len(spr_story_text.split(' ')))
    sum_data['recall_word_count'].append(len(spr_recall.split(' ')))
    
outname = 'summary_'+version.split('_end')[0]+'_n'+n_size+'_rate-importance.xlsx'
xf.write_dict_samelen(sum_data,outname)

cyoa_monthy-free_end2_xli239.txt
['1', '2', '3', '13_2', '770', '771', '772', '773', '774', '790']


'Excel file saved'

In [3]:
#3 getting subject specific rating files (story, recall, causal)

import xianfunc as xf
subjnum = 1 #int(n_size)     
sumfile = './summary_monthy-free_n1_rate-importance.xlsx'
mapfile = './monthy_map.xlsx'
out_rootdir = '../rate_importance/rate_free/' 
exclude = []#2,3,4,6]

for ind in range(0,subjnum):
    #get subj story info ('scene_lab','event_lab','scene_text') from  map_cols(scene_lab,event_lab,story) & sum_cols(sub,ID,story_route) 
    subj,data = xf.cyoa_subj_paths(mapfile,sumfile,subj_ind=ind,map_cols=[2,1,3],sum_cols=[1,2,7])
    if ind+1 not in exclude: 
        print(subj)
        #group data by events while singling out choice options
        cutted,cut_by = xf.group_cut(data['scene_text'],data['scene_label'],'_')
        group_by_new = xf.group_by_both_list(data['event_label'],cut_by)
        data['event_num'] = group_by_new
        grouped_data,n_groups = xf.group_dict(data,'event_num')
        simp,full = xf.merge_ioflist_in_data(grouped_data)

        #output the subject-specific story events file
        outfile = out_rootdir +subj+'_rate-events.xlsx' 
        head = ['event','event_lab','scenes','story_texts','rate_importance','rate_saliency']
        conv = ['event_num','event_label','scene_label','scene_text']
        conv2head_ind = [0,1,2,3]
        form = [[4],[60],[3,4],[2,3]] #col,width, col-wrap, col-hide
        xf.write_dict_samelen(simp,outfile, header=head, convert=conv, dict_ind=conv2head_ind, set_format=form)
        
print('Done')

XLRDError: Excel xlsx file; not supported